In [568]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

from sklearn.metrics import classification_report, roc_auc_score, roc_curve, make_scorer, confusion_matrix, \
    recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, GroupKFold
from sklearn.metrics import auc, precision_recall_curve,accuracy_score
from sklearn.utils import shuffle
from sklearn import tree, svm, naive_bayes, neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder,LabelEncoder
from scipy import interp
import matplotlib.pyplot as plt


In [3]:
clfs = {'svm': svm.SVC(probability=True),
        'decision_tree': tree.DecisionTreeClassifier(),
        'naive_gaussian': naive_bayes.GaussianNB(),
        'naive_mul': naive_bayes.MultinomialNB(),
        'K_neighbor': neighbors.KNeighborsClassifier(),
        'bagging_knn': BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5, max_features=0.5),
        'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5, max_features=0.5),
        'random_forest': RandomForestClassifier(n_estimators=50),
        'adaboost': AdaBoostClassifier(n_estimators=50),
        'gradient_boost': GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0)
        }

#### Measure accuracy

#### Confusion matrix generatioon

#### loop through classifiers

#### Training and testing data collecting

#### response variable -> true or false
#### independent variable -> numerical and/or sequence

In [10]:
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = np.array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]
['cold']


In [129]:
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(aminoAcidCodes)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)


In [131]:
onehot_encoded.transform([[0]])

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [347]:
onehot_encoded.transform([[aminoAcidCodes.index("ALA")]])

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [11]:
le =LabelEncoder()
le.fit([1, 2, 2, 6])
le.transform([1, 1, 2, 6]) 
le.inverse_transform([0, 0, 1, 2])


array([1, 1, 2, 6])

#### Data retrieving from MySQL

In [12]:
import pandas as pd
import numpy as np
import time
import pymysql
from sshtunnel import SSHTunnelForwarder
import matplotlib.pyplot as plt
mysql_configure = pd.read_csv("Y:/Yuan/temp/mysql_connection.csv",index_col=0)
sql_hostname = mysql_configure.loc["sql_hostname",]["value"]
sql_username = mysql_configure.loc["sql_username",]["value"]
sql_password = mysql_configure.loc["sql_password",]["value"]
sql_main_database = mysql_configure.loc["sql_main_database",]["value"]
sql_port = mysql_configure.loc["sql_port",]["value"]
ssh_host = mysql_configure.loc["ssh_host",]["value"]
ssh_user = mysql_configure.loc["ssh_user",]["value"]
ssh_password = mysql_configure.loc["ssh_password",]["value"]
ssh_port = mysql_configure.loc["ssh_port",]["value"]

In [107]:
peptidasesList = pd.read_csv("Y:/Yuan/temp/MCSA_EC3.4_peptidases.csv") #for big machine
#peptidasesList = pd.read_csv("/Volumes/Lab_Public/Yuan/temp/MCSA_EC3.4_peptidases.csv") #for mac
peptidasesList = peptidasesList[peptidasesList.iloc[:,4] == "residue"]
peptidasesList = peptidasesList.reset_index(drop=True)

In [108]:
peptidasesList.iloc[0:5,]

,M-CSA ID,Uniprot IDs,PDB,EC,residue/reactant/product/cofactor,PDB code,chain/kegg compound,resid/chebi id,function location/name,role,role type,role group
0,M0587,P00727,1lam,3.4.11.1,residue,Lys,A,262.0,side_chain,electrostatic stabiliser,spectator,electrostatic interaction
1,M0587,P00727,1lam,3.4.11.1,residue,Arg,A,336.0,side_chain,electrostatic stabiliser,spectator,electrostatic interaction
2,M0167,Q01693,1lok,3.4.11.10,residue,His,A,97.0,side_chain,metal ligand,interaction,None
3,M0167,Q01693,1lok,3.4.11.10,residue,His,A,97.0,side_chain,metal ligand,interaction,None
4,M0167,Q01693,1lok,3.4.11.10,residue,His,A,97.0,side_chain,metal ligand,interaction,None


In [158]:
group_by = peptidasesList.groupby(["PDB","chain/kegg compound","resid/chebi id","PDB code"])

In [178]:
unique_pos = peptidasesList[["PDB","chain/kegg compound","resid/chebi id","PDB code"]].drop_duplicates()

In [184]:
unique_pos.iloc[:,3] = unique_pos.iloc[:,3].str.upper()

In [236]:
unique_pos[(unique_pos.PDB =="1lam") & (unique_pos["chain/kegg compound"]=="A")]

,PDB,chain/kegg compound,resid/chebi id,PDB code
0,1lam,A,262.0,LYS
1,1lam,A,336.0,ARG


In [353]:
def n_nearest_neighbour(PDB_ID,Chain,res_ID,n):
    with SSHTunnelForwarder(
        (ssh_host, int(ssh_port)),
        ssh_username=ssh_user,
        ssh_password=ssh_password,
        remote_bind_address=('127.0.0.1', int(sql_port))) as tunnel:
        print('SSH connected')
        conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                passwd=sql_password, db=sql_main_database,
                port=tunnel.local_bind_port)
        try:
            with conn as cursor: #auto commit; no close() called
                with cursor: # close() called here
                    sql_select = "Select da.* FROM pdbdb.`Distance_angle2.0` da "
                    sql_where = "WHERE da.pdbID = \"{}\" and da.chain = \"{}\" and da.ID_1= \"{}\";".format(PDB_ID,Chain,res_ID)  
                    sql = sql_select+sql_where
                    data = pd.read_sql_query(sql, conn)
            return(data.groupby(["ID"]).first().sort_values("Distance").iloc[1:(n+1),])
        except Exception as e: # catch exceptions
            print("~~~~~~~~~~~~~~errors!!!")
            print(e)
        finally:
            if conn:
                conn.close()


In [54]:
data.groupby(["ID"]).first().sort_values("Distance").iloc[1:6,]

,pdbID,chain,ID_1,Res_1,ID_2,Res_2,Distance,Angle
ID,,,,,,,,
1022023,1a16,A,271,ASP,406,GLU,4.132360,127.382601
1021846,1a16,A,271,ASP,229,TYR,4.706969,118.528674
1021977,1a16,A,271,ASP,360,VAL,4.909811,132.652829
1021998,1a16,A,271,ASP,381,THR,5.028806,130.433577
1022025,1a16,A,271,ASP,408,ASP,5.434829,72.390018


In [119]:
def One_residue_retrieval(residue_1,PDB_ID,Chain):
    data, data_1, data_2 = None, None, None
    with SSHTunnelForwarder(
        (ssh_host, int(ssh_port)),
        ssh_username=ssh_user,
        ssh_password=ssh_password,
        remote_bind_address=('127.0.0.1', int(sql_port))) as tunnel:
            print('SSH connected')
            conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                    passwd=sql_password, db=sql_main_database,
                    port=tunnel.local_bind_port)
            try:
                with conn as cursor: #auto commit; no close() called
                    with cursor: # close() called here
                        sql_select = "Select da.* FROM pdbdb.`Distance_angle2.0` da "
                        sql_where = "WHERE da.pdbID = \"{}\" and da.chain=\"{}\" and da.ID_1 = {};".format(PDB_ID,Chain,residue_1)    
                        sql = sql_select+sql_where
                        data_1 = pd.read_sql_query(sql, conn)
                        data = data_1
                    
            except Exception as e: # catch exceptions
                print("~~~~~~~~~~~~~~")
                print(e)
            finally:
                if conn:
                    conn.close()
    return(data)

In [295]:
result_step1 = One_residue_retrieval(70,"1g2i","A")

SSH connected


In [296]:
result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard

In [298]:
result_step3 = result_step2[~result_step2.ID_2.isin([70])] # activate discard

In [301]:
result_step4 = result_step3[result_step3["Res_1"]==result_step3["Res_2"]].sample(3,random_state =1) #sample nonactive site

In [310]:
result_step4.sample(2,random_state =1).iterrows()

<generator object DataFrame.iterrows at 0x00000036762DA200>

In [442]:
pos_neg_sample = unique_pos.copy(deep=True)

In [443]:
pos_neg_sample["type"] = "POS"

In [444]:
for i in range(0,308):
    result_step1 = One_residue_retrieval(pos_neg_sample.iloc[i,2],pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1])
    result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard
    result_step3 = result_step2[~result_step2.ID_1.isin(unique_pos[(unique_pos.PDB ==pos_neg_sample.iloc[i,0]) & (unique_pos["chain/kegg compound"]==pos_neg_sample.iloc[i,1])])] # activate discard
    if(result_step3[result_step3["Res_1"]==result_step3["Res_2"]].shape[0]>1):
        print(i,2)
        result_step4 = result_step3[result_step3["Res_1"]==result_step3["Res_2"]].sample(2,random_state =1) #sample nonactive site
        for row in result_step4.iterrows():
            pos_neg_sample = pos_neg_sample.append({"PDB":row[1][0],"chain/kegg compound":row[1][1],"resid/chebi id":row[1].ID_2,"PDB code":row[1][5],"type":"NEG"},ignore_index=True)
    elif result_step3[result_step3["Res_1"]==result_step3["Res_2"]].shape[0] == 1:
        print(i,1)
        result_step4 = result_step3[result_step3["Res_1"]==result_step3["Res_2"]]
        print(result_step4)
        pos_neg_sample = pos_neg_sample.append({"PDB":result_step4.iloc[0,0],"chain/kegg compound":result_step4.iloc[0,1],"resid/chebi id":result_step4.iloc[0,4],"PDB code":result_step4.iloc[0,5],"type":"NEG"},ignore_index=True)

SSH connected
0 2
SSH connected
1 2
SSH connected
2 2
SSH connected
3 2
SSH connected
4 2
SSH connected
5 2
SSH connected
6 2
SSH connected
7 2
SSH connected
8 2
SSH connected
9 2
SSH connected
10 2
SSH connected
11 2
SSH connected
12 2
SSH connected
13 2
SSH connected
14 2
SSH connected
15 2
SSH connected
16 2
SSH connected
17 2
SSH connected
18 2
SSH connected
19 2
SSH connected
20 2
SSH connected
21 2
SSH connected
22 2
SSH connected
23 2
SSH connected
24 2
SSH connected
25 2
SSH connected
26 2
SSH connected
27 2
SSH connected
28 2
SSH connected
29 2
SSH connected
30 2
SSH connected
31 2
SSH connected
32 2
SSH connected
33 2
SSH connected
34 2
SSH connected
35 2
SSH connected
36 2
SSH connected
37 2
SSH connected
38 2
SSH connected
39 2
SSH connected
40 2
SSH connected
41 2
SSH connected
42 2
SSH connected
43 2
SSH connected
44 2
SSH connected
45 2
SSH connected
46 2
SSH connected
47 2
SSH connected
48 2
SSH connected
49 2
SSH connected
50 2
SSH connected
51 2
SSH connected
52 2
SSH

In [357]:
temp = n_nearest_neighbour("1a16","A",391,5)

SSH connected


In [363]:
temp.iloc[:,5:8].values.flatten()

array(['ALA', 4.75679454897399, 110.394827599361, 'PRO', 5.30428484583728,
       93.0963106995375, 'ALA', 5.89927360087185, 78.7414953050204, 'ARG',
       7.48509127567528, 11.3386927976814, 'GLU', 8.44035832177195,
       89.3588400095284], dtype=object)

In [364]:
pos_neg_sample_neighbor = pos_neg_sample.copy(deep=True)

##### Because of storing list in pandas datafram is tricky, another list will be created to stores neighbour info

In [448]:
neighbour_information_list = []
for i in range(0,886):
    neigh = n_nearest_neighbour(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],5).iloc[:,5:8].values.flatten()
    neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))
    

SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected


2019-10-28 21:35:21,539| ERROR   | Could not establish connection from ('127.0.0.1', 57920) to remote side of the tunnel


SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH co

SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH connected
SSH co

In [421]:
temp = n_nearest_neighbour(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],5).iloc[:,5:8].values.flatten()

SSH connected


In [450]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    for element in each_res:
        if isinstance(element,str):
            one_res_data.extend(onehot_encoded.transform([[aminoAcidCodes.index(element)]])[0].tolist())
        else:
            one_res_data.append(element)
    train_dataset.append(one_res_data)

In [465]:
temp = np.vstack(np.asarray(train_dataset[0:10]))
print(temp.shape)

(10, 142)


In [466]:
clfs["svm"].fit(temp,np.array([1,1,1,1,1,2,2,2,2,2]))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [467]:
clfs["svm"].predict(np.vstack(np.asarray(train_dataset[10:20])))

array([2, 1, 1, 2, 2, 2, 2, 2, 2, 2])

In [474]:
conf_mat = confusion_matrix(clfs["svm"].predict(np.vstack(np.asarray(train_dataset[10:20]))),[2 for i in range(10)])

In [475]:
conf_mat / 10

array([[ 0. ,  0.2],
       [ 0. ,  0.8]])

In [521]:
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<100):
        wrong_sample.append(i)

In [509]:
len(pos_neg_sample_neighbor[pos_neg_sample_neighbor.type == "POS"])

308

In [514]:
y_label =[1 for x in range(0,308)]

In [515]:
y_label.extend([2 for i in range(0,578)])

In [552]:
len(y_label)

886

In [560]:
final_y_label = [y_label[i] for i in range(0,886) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label)

In [561]:
final_train_dataset = [train_dataset[i] for i in range(0,886) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset)

In [583]:
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(shuffle_train_dataset,shuffle_y_label):
    clfs["gradient_boost"].fit(shuffle_train_dataset[train],shuffle_y_label[train])
    conf_mat = confusion_matrix(clfs["gradient_boost"].predict(shuffle_train_dataset[test]),shuffle_y_label[test])
    print(conf_mat.ravel())
    print(accuracy_score(clfs["gradient_boost"].predict(shuffle_train_dataset[test]),shuffle_y_label[test]))

[ 13  38  84 155]
0.579310344828
[ 17  37  80 156]
0.596551724138
[ 20  43  76 149]
0.586805555556


In [571]:
import json
import requests
url = 'http://3dcons.cnb.csic.es/pssm_json/1R44/A'
file = requests.get(url)

In [572]:
json_decoded = json.loads(file.content)
json_decoded[0]['iter']['2']['pssm']

clfs = {'svm': svm.SVC(probability=True),
        'decision_tree': tree.DecisionTreeClassifier(),
        'naive_gaussian': naive_bayes.GaussianNB(),
        'naive_mul': naive_bayes.MultinomialNB(),
        'K_neighbor': neighbors.KNeighborsClassifier(),
        'bagging_knn': BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5, max_features=0.5),
        'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5, max_features=0.5),
        'random_forest': RandomForestClassifier(n_estimators=50),
        'adaboost': AdaBoostClassifier(n_estimators=50),
        'gradient_boost': GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0)
        }